In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/xj-first-sentence/xj_first_sentence_train.csv
/kaggle/input/xj-first-sentence/xj_first_sentence_val.csv
/kaggle/input/xj-first-sentence/xj_first_sentence_test.csv


In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, BertModel
from transformers import AdamW, get_scheduler
import torch.nn.functional as F
from tqdm import tqdm
import torch.nn as nn
from sklearn.metrics import roc_auc_score


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
train_df = pd.read_csv('/kaggle/input/xj-first-sentence/xj_first_sentence_train.csv')
val_df = pd.read_csv('/kaggle/input/xj-first-sentence/xj_first_sentence_val.csv')
test_df = pd.read_csv('/kaggle/input/xj-first-sentence/xj_first_sentence_test.csv')


In [5]:
X_train = train_df['xi'] + " " + train_df['xj']
X_val = val_df['xi'] + " " + val_df['xj']

# Labels
y_train = train_df['LLM_model'].factorize()[0]
y_val = val_df['LLM_model'].factorize()[0]



In [6]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
def encode_data(texts, tokenizer, max_len=128):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=max_len, return_tensors="pt")


In [8]:
encoded_train = encode_data(X_train, tokenizer, max_len=128)
encoded_val = encode_data(X_val, tokenizer, max_len=128)


In [9]:
X_train_bert = encoded_train['input_ids']
attention_mask_train = encoded_train['attention_mask']
X_val_bert = encoded_val['input_ids']
attention_mask_val = encoded_val['attention_mask']

In [10]:
class BertDataset(Dataset):
    def __init__(self, X, attention_mask, y):
        self.X = X
        self.attention_mask = attention_mask
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.attention_mask[idx], self.y[idx]


In [11]:
train_dataset_bert = BertDataset(X_train_bert, attention_mask_train, y_train)
val_dataset_bert = BertDataset(X_val_bert, attention_mask_val, y_val)
train_loader_bert = DataLoader(train_dataset_bert, batch_size=64, shuffle=True)
val_loader_bert = DataLoader(val_dataset_bert, batch_size=64)


In [12]:
class EnhancedModel(nn.Module):
    def __init__(self, num_classes):
        super(EnhancedModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')  
        self.dropout = nn.Dropout(p=0.2)  
        self.fc1 = nn.Linear(self.bert.config.hidden_size, 512)  
        self.fc2 = nn.Linear(512, 256)  
        self.classifier = nn.Linear(256, num_classes) 

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]  
        x = self.dropout(pooled_output)
        x = F.relu(self.fc1(x))  
        x = F.relu(self.fc2(x))  
        x = self.dropout(x)
        return self.classifier(x)


In [13]:
num_classes = len(set(y_train))
bert_model = EnhancedModel(num_classes).to(device)
epochs = 20  
#1e-5
optimizer = AdamW(bert_model.parameters(), lr=1e-5, weight_decay=1e-4)  
num_training_steps = len(train_loader_bert) * epochs  
num_warmup_steps = int(0.1 * num_training_steps)  
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)



config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
def train_bert_model(model, train_loader, val_loader, optimizer, scheduler):
    #patience = 5  # Number of epochs to wait before stopping
    epochs_no_improve = 0
    best_val_loss = float('inf')
    
    for param in model.bert.parameters():
        param.requires_grad = False


    for epoch in range(epochs):
        model.train()
        total_loss, total_correct = 0, 0

        for X_batch, attention_mask, y_batch in tqdm(train_loader):
            X_batch, attention_mask, y_batch = X_batch.to(device), attention_mask.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch, attention_mask=attention_mask)  
            loss = F.cross_entropy(outputs, y_batch)
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == y_batch).sum().item()

        accuracy = total_correct / len(train_loader.dataset)
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}, Accuracy: {accuracy}")

        # Validate
        val_loss, val_accuracy, val_auc = validate_bert_model(model, val_loader)
        print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}, AUC-ROC: {val_auc}")

def validate_bert_model(model, val_loader):
    model.eval()
    total_loss, total_correct = 0, 0
    all_targets = []
    all_probs = []

    with torch.no_grad():
        for X_batch, attention_mask, y_batch in val_loader:
            X_batch, attention_mask, y_batch = X_batch.to(device), attention_mask.to(device), y_batch.to(device)
            outputs = model(X_batch, attention_mask=attention_mask) 
            loss = F.cross_entropy(outputs, y_batch)

            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == y_batch).sum().item()

            all_targets.extend(y_batch.cpu().numpy())
            all_probs.extend(F.softmax(outputs, dim=1).cpu().numpy()) 

    val_accuracy = total_correct / len(val_loader.dataset)
    val_auc = roc_auc_score(all_targets, all_probs, multi_class='ovr')  

    return total_loss / len(val_loader), val_accuracy, val_auc

In [15]:
train_bert_model(bert_model, train_loader_bert, val_loader_bert, optimizer, lr_scheduler)


100%|██████████| 63/63 [00:12<00:00,  5.20it/s]


Epoch 1, Loss: 1.6104492478900485, Accuracy: 0.20529470529470528
Validation Loss: 1.6108851432800293, Validation Accuracy: 0.194, AUC-ROC: 0.4834525


100%|██████████| 63/63 [00:11<00:00,  5.36it/s]


Epoch 2, Loss: 1.6102674575079055, Accuracy: 0.20654345654345654
Validation Loss: 1.6105904877185822, Validation Accuracy: 0.208, AUC-ROC: 0.4712250000000001


100%|██████████| 63/63 [00:12<00:00,  5.22it/s]


...

Epoch 18, Loss: 1.5943288746334257, Accuracy: 0.25274725274725274
Validation Loss: 1.6184746325016022, Validation Accuracy: 0.208, AUC-ROC: 0.48436


100%|██████████| 63/63 [00:13<00:00,  4.59it/s]


Epoch 19, Loss: 1.5961455712242731, Accuracy: 0.24525474525474525
Validation Loss: 1.6185636520385742, Validation Accuracy: 0.206, AUC-ROC: 0.4847


100%|██████████| 63/63 [00:13<00:00,  4.58it/s]


Epoch 20, Loss: 1.5942936549111018, Accuracy: 0.247002997002997
Validation Loss: 1.6185556948184967, Validation Accuracy: 0.208, AUC-ROC: 0.48464000000000007


In [16]:
def prepare_test_data(test_df, tokenizer, max_len=100):
    X_test = test_df['xi'] + " " + test_df['xj']  
    y_test = test_df['LLM_model'].factorize()[0]  

    encoded_test = encode_data(X_test, tokenizer, max_len)
    X_test_bert = encoded_test['input_ids']
    attention_mask_test = encoded_test['attention_mask']

    test_dataset_bert = BertDataset(X_test_bert, attention_mask_test, y_test)
    return DataLoader(test_dataset_bert, batch_size=64), y_test  

test_loader_bert, y_test = prepare_test_data(test_df, tokenizer, max_len=100)

def evaluate_test_data(model, test_loader):
    model.eval()
    total_loss, total_correct = 0, 0
    all_targets = []
    all_probs = []

    with torch.no_grad():
        for X_batch, attention_mask, y_batch in test_loader:
            X_batch, attention_mask, y_batch = X_batch.to(device), attention_mask.to(device), y_batch.to(device)
            outputs = model(X_batch, attention_mask=attention_mask)  
            loss = F.cross_entropy(outputs, y_batch)

            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == y_batch).sum().item()
            all_targets.extend(y_batch.cpu().numpy())
            all_probs.extend(F.softmax(outputs, dim=1).cpu().numpy())

    test_accuracy = total_correct / len(test_loader.dataset)
    test_loss = total_loss / len(test_loader)
    test_auc = roc_auc_score(all_targets, all_probs, multi_class='ovr')

    return test_loss, test_accuracy, test_auc

In [17]:
test_loss, test_accuracy, test_auc = evaluate_test_data(bert_model, test_loader_bert)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}, Test AUC-ROC: {test_auc}")

Test Loss: 1.602718785405159, Test Accuracy: 0.2155688622754491, Test AUC-ROC: 0.5524759141749587
